In [1]:
from google.cloud import bigquery, storage
import csv
import io

# Initialize the BigQuery client for the source project
bq_client = bigquery.Client(project='yhcr-prd-bradfor-bia-core')

# Initialize the Cloud Storage client for the target project
storage_client = storage.Client(project='yhcr-prd-bradfor-bia-core')

# Define the source project, dataset
source_project_id = 'yhcr-prd-bradfor-bia-core'
source_dataset_id = 'CB_1760'

# Define the destination GCS bucket
bucket_name = 'yhcr-prd-phm-bia-core-data-landing-bradford'

# List of tables to process
table_list = [
    'Type_2_Diabetes_Annual_Review_All_Ordered_10M_1','Type_2_Diabetes_Annual_Review_All_Ordered_10M_2'
]

# Process each table in the list
for source_table_id in table_list:
    # Construct the fully-qualified table ID
    table_id = f"{source_project_id}.{source_dataset_id}.{source_table_id}"
    
    # Define the destination GCS file path (using table name as the file name)
    gcs_file_path = f"JA/{source_table_id}.csv"  # Folder within the bucket

    # Define the query to fetch data from the source BigQuery table
    query = f"SELECT * FROM `{table_id}`"
    
    # Run the query
    query_job = bq_client.query(query)
    
    # Wait for the query to complete and retrieve the result
    query_result = query_job.result()  # Ensure we wait for the query to complete and retrieve results

    # Check if schema is available
    if query_result.schema is None:
        raise ValueError(f"Schema is not available for table {source_table_id}. Please check the query and table.")
    
    # Prepare the GCS bucket object (assuming the bucket exists in the target project)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(gcs_file_path)

    # Create a memory buffer to hold the CSV data
    csv_buffer = io.StringIO()
    csv_writer = csv.writer(csv_buffer)

    # Write headers (column names) to the CSV buffer
    headers = [field.name for field in query_result.schema]  # Access schema from query_result
    csv_writer.writerow(headers)

    # Write the query results to the CSV buffer
    for row in query_result:  # Now use query_result instead of query_job
        csv_writer.writerow(row.values())

    # Upload the CSV data from the memory buffer to GCS
    csv_buffer.seek(0)  # Move the cursor to the beginning of the buffer
    blob.upload_from_file(csv_buffer, content_type='text/csv')

    print(f"Data exported to gs://{bucket_name}/{gcs_file_path} for table {source_table_id}")

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


BadRequest: 400 PUT https://storage.googleapis.com/upload/storage/v1/b/yhcr-prd-phm-bia-core-data-landing-bradford/o?uploadType=resumable&upload_id=AHMx-iHG1U6Ty1AnbWVy82j0pA9cVIOuTdjhVgvAJraWGol1M9tNkaOWCvRWRIYJZi5fEb57HIufFWzB-7deRKhXu9NFx4hHgCSUcOg-0Y5NQFQ: Invalid request.  There were 104873984 byte(s) (or more) in the request body.  There should have been 104857600 byte(s) (starting at offset 0 and ending at offset 104857599) according to the Content-Range header.: ('Request failed with status code', 400, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.PERMANENT_REDIRECT: 308>)